<a href="https://colab.research.google.com/github/saurograndi/nlp-t5-summarizer/blob/main/mT5-cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization with T5

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install datasets
!pip install nltk
!pip install rouge_score
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=2a937c4151a57e4be8fafb6bb01eb803530cc13e63ec6822cd3dee9747242e4b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 886.4 kB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [3]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('token'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import datasets

cnn = datasets.load_dataset("cnn_dailymail", "3.0.0")
cnn

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
def get_samples(dataset, num_samples=100, seed=42):
    train_sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    valid_sample = dataset["validation"].shuffle(seed=seed).select(range(int(num_samples/6.7)))
    test_sample = dataset["test"].shuffle(seed=seed).select(range(int(num_samples/6.7)))
    return datasets.DatasetDict({"train":train_sample, "valid":valid_sample, "test":test_sample})

In [6]:
cnn = get_samples(cnn, num_samples=1000)
cnn

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1000
    })
    valid: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 149
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 149
    })
})

In [7]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> article: {example['article']}'")
        print(f"\n'>> highlights: {example['highlights']}'")


show_samples(cnn, num_samples=1)


'>> article: By . Nick Enoch . Star trails sweep over the Giant’s Causeway in Northern Ireland, dust clouds are moulded into colossal arrangements by cosmic radiation thousands of light years away and a bright meteor races across the night sky passing over Indonesia’s smoke-spewing Mount Bromo. These are just some of the incredible photos which have been shortlisted in the 2014 Astronomy Photographer of the Year competition. The contest, run by the Royal Observatory Greenwich in association with BBC Sky at Night Magazine, is now in its sixth year - and a record number of entries from more than 2,500 enthusiastic amateurs and professional photographers have poured in from around the world. Centre of the Heart Nebula by Ivan Eder, Hungary. Situated 7,500 light years away in the W-shaped constellation of Cassiopeia, the Heart Nebula is a vast region of glowing gas, energised by a cluster of young stars at its centre. The image depicts the central region, where dust clouds are being erode

## Preprocessing

In [8]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_cnn = cnn.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [10]:
tokenized_cnn = tokenized_cnn.remove_columns(['id'])

In [11]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

## Evaluation

In [13]:
import numpy as np
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Training

In [14]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Load T5 with TFAutoModelForSeq2SeqLM:

Convert your datasets to the tf.data.Dataset format with prepare_tf_dataset():

In [15]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_cnn["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_cnn["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Transformers models all have a default task-relevant loss function

In [16]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [20]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

#callbacks = [metric_callback]
callbacks = None

In [21]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

Epoch 1/3
62/62 [==============================] - 47s 344ms/step - loss: 2.4108 - val_loss: 2.2179
Epoch 2/3
62/62 [==============================] - 18s 289ms/step - loss: 2.1707 - val_loss: 2.0810
Epoch 3/3
62/62 [==============================] - 18s 283ms/step - loss: 2.1146 - val_loss: 2.0517


## Inference

In [35]:
text = "summarize: Questa è una giornata per recuperare un terreno che non abbiamo saputo calpestare nel modo giusto”. Gianni Cuperlo presenta così la lunga giornata del convegno “La parola Pace – L’utopia che deve farsi realtà”, promosso a Milano dalla sua associazione Promessa democratica. Sul palco si alternano decine di dirigenti del Pd, fino alla segretaria Elly Schlein. E l’evento, oltre ai validi contributi sul Medio Oriente (tra cui quelli di Lucia Annunziata e Domenico Quirico), suona soprattutto come un netto cambio di atteggiamento del partito nei confronti della guerra in Ucraina. Persino un mea culpa, a giudicare dai toni di alcuni contributi. Non solo dall’ala più a sinistra, ma pure da Base Riformista, la corrente guidata da Lorenzo Guerini, seduto in prima fila per tutta la mattinata."

In [36]:
inputs = tokenizer(text, return_tensors="tf").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Gianni Cuperlo presenta cos la lunga giornata del convegno “La parola Pace – L’utopia che deve farsi realtà” Milano dalla sua associazione Promessa democratica.'